In [1]:
import backtrader as bt
import pandas as pd
import numpy as np

# 您提供的LLT计算函数
def calculate_llt(prices, alpha=0.05):
    n = len(prices)
    llt = np.zeros(n)
    if n >= 1:
        llt[0] = prices[0]
    if n >= 2:
        llt[1] = prices[1]
    
    a1 = alpha - (alpha**2) / 4
    a2 = (alpha**2) / 2
    a3 = alpha - 3 * (alpha**2) / 4
    a4 = 2 * (1 - alpha)
    a5 = - (1 - alpha)**2
    
    for t in range(2, n):
        llt[t] = a1 * prices[t] + a2 * prices[t-1] - a3 * prices[t-2] + a4 * llt[t-1] + a5 * llt[t-2]
    
    return llt

# 您提供的信号计算函数
def llt_slope_signal(df, d: int=30, slope_window=5):
    df = df.copy()
    alpha = 2 / (d + 1)
    df["llt"] = calculate_llt(df["close"], alpha)
    df['slope'] = (df["llt"].rolling(slope_window)
                    .apply(lambda x: np.polyfit(np.arange(slope_window), x, 1)[0]))
    
    signals = pd.Series(0, index=df.index)
    signals[df['slope'] > 0] = 1
    signals[df['slope'] < 0] = -1
    
    return signals


In [68]:
class PortfolioLogger(bt.Observer):
    lines = ('value', 'cash', 'long_value', 'short_value')
    
    plotinfo = dict(
        plot=True,
        subplot=True,
        plotname='Portfolio Value',
        plotlinelabels=True
    )
    
    plotlines = dict(
        value=dict(_name='Total Value'),
        cash=dict(_name='Cash'),
        long_value=dict(_name='Long Value'),
        short_value=dict(_name='Short Value')
    )
    
    def __init__(self):
        self.long_positions = {}
        self.short_positions = {}
        
    def next(self):
        self.l.value[0] = self._owner.broker.getvalue()
        self.l.cash[0] = self._owner.broker.getcash()
        
        long_value = 0.0
        short_value = 0.0
        
        for data in self._owner.datas:
            position = self._owner.getposition(data)
            
            if position.size > 0:
                long_value += position.size * data.close[0]
            elif position.size < 0:
                short_value += position.size * data.close[0]
        
        self.l.long_value[0] = long_value
        self.l.short_value[0] = short_value
        
        self.log(f"总价值: {self.l.value[0]:.2f}, 现金: {self.l.cash[0]:.2f}, "
                 f"多头: {long_value:.2f}, 空头: {short_value:.2f}")
    
    def log(self, txt):
        date = self._owner.data.datetime.date(0)
        print(f"{date}: {txt}")

class LongShort1x1Strategy(bt.Strategy):
    params = (
        ('d', 5),                # LLT参数d
        ('slope_window', 5),      # 斜率窗口
        ('position_ratio', 1),
    )
    
    def __init__(self):
        # 预计算信号
        self.signals = llt_slope_signal(
            self.data._dataname,  # 访问原始DataFrame
            d=5,
            slope_window=self.p.slope_window
        )
        self.signals = self.signals.tz_localize(None)  # 统一日期格式
        self.portfolio_logger = PortfolioLogger(self)
    
    def next(self):
        # 获取当前日期和信号
        current_date = pd.Timestamp(self.data.datetime.date())
        
        # 获取当前信号（处理日期不匹配）
        try:
            current_signal = self.signals.loc[current_date]
        except KeyError:
            current_signal = 0
                
        if current_signal == 1:
            self.order_target_percent(target = 0.95)
            # self.order_target_size(target = 1)
        elif current_signal == -1:
            self.order_target_percent(target = -0.95)
            # self.order_target_size(target = -1)
        else:
            self.close()

    def notify_trade(self, trade):
        if trade.isclosed:
            print(f"交易结束：{trade.close_datetime()}")
            print(f"    开仓价格: {trade.price}")
            print(f"    平仓价格： ")
            print(f"    pnl: {trade.pnl}")


In [69]:
def run_backtest(data, d=30, slope_window=5, initial_cash=1_000_000):
    # 数据清洗（关键：避免绘图错误）
    data = data.replace([np.inf, -np.inf], np.nan).dropna()
    if 'volume' in data.columns:
        data['volume'] = data['volume'].clip(lower=0)  # 成交量非负
    
    # 初始化回测引擎
    cerebro = bt.Cerebro()
    cerebro.addstrategy(LongShort1x1Strategy, d=d, slope_window=slope_window)
    
    # 添加数据
    bt_data = bt.feeds.PandasData(dataname=data)
    cerebro.adddata(bt_data)
    
    # 配置回测参数
    cerebro.broker.setcash(initial_cash)
    cerebro.broker.setcommission(commission=0.001)  # 佣金0.1%
    
    # 添加绩效分析器
    cerebro.addanalyzer(bt.analyzers.SharpeRatio, _name='sharpe')
    cerebro.addanalyzer(bt.analyzers.DrawDown, _name='drawdown')
    cerebro.addanalyzer(bt.analyzers.Returns, _name='returns')
    
    # 运行回测
    print(f"初始资金: {cerebro.broker.getvalue():.2f}")
    results = cerebro.run()
    final_value = cerebro.broker.getvalue()
    print(f"最终资金: {final_value:.2f}")
    
    # 输出绩效指标（处理可能的None值）
    strat = results[0]
    returns = strat.analyzers.returns.get_analysis()
    sharpe = strat.analyzers.sharpe.get_analysis()
    drawdown = strat.analyzers.drawdown.get_analysis()
    
    print(f"年化收益率: {returns.get('rnorm100', 0):.2f}%")
    print(f"夏普比率: {sharpe.get('sharperatio', 0):.2f}")
    print(f"最大回撤: {drawdown.get('max', {}).get('drawdown', 0):.2f}%")


    return results

def get_price(symbol, start_date, end_date):
    pro = pro_api()

    price_df = pro.index_daily(
        ts_code=symbol,
        start_date=start_date.strftime("%Y%m%d"),
        end_date=end_date.strftime("%Y%m%d"),
    )

    price_df = (
        price_df.rename({"trade_date": "date", "ts_code": "asset"}, axis=1)
        .sort_values("date", ascending=True)
        .set_index("date")
    )

    return price_df

In [70]:
start = datetime.date(2013, 1, 1)
end = datetime.date(2024,12, 31)

prices = get_price("000001.SH", start, end)
prices.index = pd.to_datetime(prices.index)
result = run_backtest(prices, d = 5)

初始资金: 1000000.00


RecursionError: maximum recursion depth exceeded

In [15]:
pd.options.display.max_rows = 300
result[0].signals.head(100)

date
2013-01-04    0
2013-01-07    0
2013-01-08    0
2013-01-09    0
2013-01-10    1
2013-01-11   -1
2013-01-14   -1
2013-01-15    1
2013-01-16    1
2013-01-17    1
2013-01-18    1
2013-01-21    1
2013-01-22    1
2013-01-23    1
2013-01-24    1
2013-01-25   -1
2013-01-28   -1
2013-01-29    1
2013-01-30    1
2013-01-31    1
2013-02-01    1
2013-02-04    1
2013-02-05    1
2013-02-06    1
2013-02-07    1
2013-02-08    1
2013-02-18   -1
2013-02-19   -1
2013-02-20   -1
2013-02-21   -1
2013-02-22   -1
2013-02-25   -1
2013-02-26   -1
2013-02-27   -1
2013-02-28   -1
2013-03-01    1
2013-03-04    1
2013-03-05    1
2013-03-06   -1
2013-03-07   -1
2013-03-08    1
2013-03-11    1
2013-03-12   -1
2013-03-13   -1
2013-03-14   -1
2013-03-15   -1
2013-03-18   -1
2013-03-19   -1
2013-03-20   -1
2013-03-21    1
2013-03-22    1
2013-03-25    1
2013-03-26    1
2013-03-27    1
2013-03-28   -1
2013-03-29   -1
2013-04-01   -1
2013-04-02   -1
2013-04-03   -1
2013-04-08   -1
2013-04-09   -1
2013-04-10   -1
201

In [ ]:
start = datetime.date(2005, 9, 6)
end = datetime.date(2013, 6, 28)

prices = get_price("000001.SH", start, end)
prices.index = pd.to_datetime(prices.index)

result = run_simple_backtest(prices)


初始资金: 100000.00
增加空头: 2005-10-10, 价格=1138.95, 数量=0
2005-10-10 - 多头:0, 空头:1, 现金:100000.00
2005-10-11 - 多头:0, 空头:1, 现金:101136.81
2005-10-12 - 多头:0, 空头:1, 现金:101136.81
2005-10-13 - 多头:0, 空头:1, 现金:101136.81
2005-10-14 - 多头:0, 空头:1, 现金:101136.81
2005-10-17 - 多头:0, 空头:1, 现金:101136.81
2005-10-18 - 多头:0, 空头:1, 现金:101136.81
2005-10-19 - 多头:0, 空头:1, 现金:101136.81
2005-10-20 - 多头:0, 空头:1, 现金:101136.81
2005-10-21 - 多头:0, 空头:1, 现金:101136.81
增加空头: 2005-10-24, 价格=1141.17, 数量=0
2005-10-24 - 多头:0, 空头:2, 现金:101136.81
2005-10-25 - 多头:0, 空头:2, 现金:102275.84
2005-10-26 - 多头:0, 空头:2, 现金:102275.84
2005-10-27 - 多头:0, 空头:2, 现金:102275.84
2005-10-28 - 多头:0, 空头:2, 现金:102275.84
减少空头: 2005-10-31, 价格=1092.82, 数量=0
2005-10-31 - 多头:0, 空头:1, 现金:102275.84
2005-11-01 - 多头:0, 空头:1, 现金:101183.07
2005-11-02 - 多头:0, 空头:1, 现金:101183.07
2005-11-03 - 多头:0, 空头:1, 现金:101183.07
2005-11-04 - 多头:0, 空头:1, 现金:101183.07
2005-11-07 - 多头:0, 空头:1, 现金:101183.07
2005-11-08 - 多头:0, 空头:1, 现金:101183.07
增加空头: 2005-11-09, 价格=1108.15, 数量=0
2005-11-

In [ ]:
start = datetime.date(2005, 9, 6)
end = datetime.date(2013, 6, 28)

prices = get_price("000001.SH", start, end)
prices.index = pd.to_datetime(prices.index)

result = run_simple_backtest(prices)


初始资金: 100000.00
增加空头: 2005-10-10, 价格=1138.95, 数量=0
2005-10-10 - 多头:0, 空头:1, 现金:100000.00
2005-10-11 - 多头:0, 空头:1, 现金:101136.81
2005-10-12 - 多头:0, 空头:1, 现金:101136.81
2005-10-13 - 多头:0, 空头:1, 现金:101136.81
2005-10-14 - 多头:0, 空头:1, 现金:101136.81
2005-10-17 - 多头:0, 空头:1, 现金:101136.81
2005-10-18 - 多头:0, 空头:1, 现金:101136.81
2005-10-19 - 多头:0, 空头:1, 现金:101136.81
2005-10-20 - 多头:0, 空头:1, 现金:101136.81
2005-10-21 - 多头:0, 空头:1, 现金:101136.81
增加空头: 2005-10-24, 价格=1141.17, 数量=0
2005-10-24 - 多头:0, 空头:2, 现金:101136.81
2005-10-25 - 多头:0, 空头:2, 现金:102275.84
2005-10-26 - 多头:0, 空头:2, 现金:102275.84
2005-10-27 - 多头:0, 空头:2, 现金:102275.84
2005-10-28 - 多头:0, 空头:2, 现金:102275.84
减少空头: 2005-10-31, 价格=1092.82, 数量=0
2005-10-31 - 多头:0, 空头:1, 现金:102275.84
2005-11-01 - 多头:0, 空头:1, 现金:101183.07
2005-11-02 - 多头:0, 空头:1, 现金:101183.07
2005-11-03 - 多头:0, 空头:1, 现金:101183.07
2005-11-04 - 多头:0, 空头:1, 现金:101183.07
2005-11-07 - 多头:0, 空头:1, 现金:101183.07
2005-11-08 - 多头:0, 空头:1, 现金:101183.07
增加空头: 2005-11-09, 价格=1108.15, 数量=0
2005-11-